# Модуль Б. Разработка модели машинного обучения

## Импортирование библиотек

In [ ]:
# 
import zipfile

#
from ultralytics import YOLO

# 
import pandas as pd

#
import numpy as np

#
import matplotlib.pyplot as plt

#
import os

# 
from deepface import DeepFace





In [3]:
# назначаем основные пути
module_dir = os.getcwd()
main_dir = module_dir[:-8]
data_dir = os.path.join(main_dir, r'Module1\Data')
# логирование
print(os.path.exists(main_dir))
print(os.path.exists(data_dir))
print(os.listdir(data_dir))

True
True
['face_detection.zip', 'face_recognition.zip']


## Загрузка данных

In [4]:
# получаем пути к архивам
face_det_zip = os.path.join(data_dir, os.listdir(data_dir)[0])
face_rec_zip = os.path.join(data_dir, os.listdir(data_dir)[1])
# логирование
print(face_det_zip)
print(face_rec_zip)
print(os.path.exists(face_det_zip))
print(os.path.exists(face_rec_zip))


d:\Helper\MLBazyak\chemp\face_recognitionV2\Module1\Data\face_detection.zip
d:\Helper\MLBazyak\chemp\face_recognitionV2\Module1\Data\face_recognition.zip
True
True


In [5]:
# создаем папку с данными
module_data_path = os.path.join(module_dir, 'Data')
os.makedirs(module_data_path, exist_ok=True)
# и папки с данными для детекции и распознавания
face_det = os.path.join(module_data_path, 'face_detection')
face_rec = os.path.join(module_data_path, 'face_recognition')
os.makedirs(face_det, exist_ok=True)
os.makedirs(face_rec, exist_ok=True)

для продолжения работы, надо разархивировать данные с прошлого модуля

In [7]:
# датасет для детекции
with zipfile.ZipFile(face_det_zip, 'r') as zip:
    # выгружаем файлы из архива
    zip.extractall(face_det)

# датасет для распознавания
with zipfile.ZipFile(face_rec_zip, 'r') as zip:
    # выгружаем файлы из архива
    zip.extractall(face_rec)

print('Данные успешно разархивированны')

Данные успешно разархивированны


## Выбор алгоритма машинного обучения

**Алгоритм детекции:**

В качестве алгоритма детекции, я буду использовать предобученную модель из семейства `YOLO` - `yolov8n` (nano-версия YOLOv8). Это самая маленькая модель из линейки, обеспечивает оптимальный баланс между точностью и скоростью. Это может быть важно, для решения задач реального времени (в нашем случае - детекция лиц).

Также, модель достаточно легкая, что позволяет работать на устройствах с ограниченными вычислительными мощностями, и так как в моем распоряжении только CPU (процессор), подобная модель будет оптимальным выбором

Архитектура моделей из семейства `YOLO` - разработана для решения задачи детекции, и обрабатывает изображение за 1 проход через нейронную сеть. Это делает подобные модели быстрее по сравнению с другими подходами, такими как `R-CNN` или `Faster R-CNN`.

`YOLOv8` поставляется с предобученными весами, что позволяет использовать ее сразу, или дообучить модель на собственных данных, настроив ее на более точное решение задачи

У `YOLOv8` простой и удобный API, который позволяет быстро начать работу с моделью, интегрируя ее в свой проект

Хоть мы и берем веса `nano`, всегда есть возможность переключить на более тяжелые модели, например `medium` или `large`, так как `YOLO` легко масштабируется

По итогу, я считаю что для моей задачи данная модель - наилучший вариант, поэтому для своего решения я буду использовать именно её

------------------------------------------

**Алгоритм распознования:**

В качестве алгоритма распознавания, я буду использовать `косинусное сходство` `эмбеддингов`, извлеченных с помощью модели `Facenet512` из библиотеки `DeepFace`

**Facenet512**

Модель `Facenet512` - одна из самых популярных и точных моделей для задач распознавания лиц. Она основана на архитектуре глубоких нейронных сетей, и предобученна на больших наборах данных

Модель преобразует фотографии в `512-мерные` эмбеддинге (векторные представления), которые эффективно кодируют уникальные черты лица

`Facenet512` показывает отличные результаты на стандартных `бенчмарках`, что делает ее надежным выбором для задач распознавания

Модель извлекает эмбеддинги за 1 проход через нейронную сеть, что делает процесс быстрым и эффективным

Также, Facenet512 устойчива к различным вариациям изображений, что делает ее подходящей для работы в реальных условиях

--------------------------------------------------------------------------

**Косинусное сходство**

`Косинусное сходство` - это метрика, которая измеряет угол между двумя векторами в многомерном пространстве. Она идеально подходит для сравнения эмбеддингов, так как :
- не зависит от длины векторов
- фокусируется на направлении векторов

Метрика возвращает значение в диапазоне от `-1 до 1`, где 1 означает `полное совпадение`, а -1 - `полное различие`

Метрика проста в вычислении и интерпретации, что делает ее удобной для `задач распознавания`


## Модель детекции

In [8]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:01<00:00, 3.58MB/s]


## Модель распознавания

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms

In [10]:
import os
import shutil
import random

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 0.001
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


подготовка датасета

In [15]:
# папки с фотографиями
crop_source = './Data/face_recognition/Faces/Faces'
train = './Data/face_recognition/train'
test = './Data/face_recognition/test'

# если папки с выборками нет - создаем их
os.makedirs(train, exist_ok=True)
os.makedirs(test, exist_ok=True)

# словарь с именами людей и их фотографиями
names_dict = {}
counter = 0
#
for file_name in os.listdir(crop_source):
    name = file_name.split('_')[0]
    names_dict.setdefault(name, []).append(file_name)
    counter +=1

for name, data in tqdm(names_dict.items(), desc='Разделение фото на выборки..', unit='image'):
    random.shuffle(data)
    # процент от всех данных (80% - train, остальное - test)
    split_value = int(len(data)*0.80)
    # разделяем список на выборки
    train_files, test_files = data[:split_value], data[split_value:]

    # для каждого имени создаем папку
    os.makedirs(os.path.join(train, name), exist_ok=True)
    os.makedirs(os.path.join(test, name), exist_ok=True)

    # копируем файлы в созданные папки
    for file in train_files:
        shutil.copy(os.path.join(crop_source, file), os.path.join(train, name, file))
    for file in test_files:
        shutil.copy(os.path.join(crop_source, file), os.path.join(test, name, file))

print('Разделение на выборки завершено')

Разделение фото на выборки..: 100%|██████████| 31/31 [01:14<00:00,  2.42s/image]

Разделение на выборки завершено


Предобработка фотографий

In [ ]:
# пайплайн для обработки фотографий
transform = transforms.Compose([
    transforms.Resize((224,224)),   # изменение размера на 224х224
    transforms.ToTensor(),          # приведение к тензору
    transforms.Normalize([0.5],[0.5]) # нормализация (mean=0.5, std=0.5)
])

# загружаем папки с фотграфиями
train_dataset = ImageFolder(root=train, transform=transform)
test_dataset = ImageFolder(root=test, transform=transform)

# создаем датасеты из папок
train_loader = DataLoader()